# NC5: Phân cụm phòng ban dựa trên số lượng email.

Công ty PiMA Corp gồm 100 nhân viên được chia làm 4 phòng ban. Mỗi nhân viên có xu hướng trao đổi nhiều emails với người cùng phòng hơn người khác phòng. 
File `emails.csv` chứa một ma trận $M \in \mathbb{R}^{100 \times 100}$, trong do $M_{ij}$ là số emails nhân viên $i$ gửi tới nhân viên $j$ trong vòng $1$ tháng. **Lưu ý:** $M_{ij}$ có thề khác $M_{ji}$.

**Câu hỏi:** Liệu có thể dùng K-means để tìm ra ai thuộc phòng ban nào? Nếu không, bạn có thể đề xuất một cải biến của K-means để tìm giải bài toán trên không?

(Đề tuyển trại sinh PiMA 2021 - Phần Tự nghiên cứu)

## Trả lời:

Khó khăn khi áp dụng K-means trong trường hợp trên là các điểm dữ liệu không được cho dưới dạng một vector mà bởi một ma trận gồm thông tin về "độ giống nhau" giữa một cặp điểm. Em đề xuất sử dụng một thuật toán phân cụm là [Spectral Clustering](https://arxiv.org/pdf/0711.0189.pdf) để giải quyết bài toán có kiểu dữ liệu đầu vào đặc biệt này.

(Spectral Clustering không phải là một cải biến của K-means, nhưng là cách đã được sử dụng trong khi giải đề tuyển sinh. Một cải biến của K-means có thể giải quyết bài toán trên là [K-medoids](https://en.wikipedia.org/wiki/K-medoids).)

## Import các thư viện

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering

## Load dữ liệu

In [2]:
emails = np.genfromtxt('data/emails.csv', delimiter=',')

In [3]:
emails

array([[ 0., 14.,  5., ...,  3., 15.,  2.],
       [17.,  0.,  7., ...,  1., 17.,  8.],
       [ 3.,  4.,  0., ...,  2.,  2., 16.],
       ...,
       [ 5.,  2.,  2., ...,  0.,  2.,  5.],
       [16., 18.,  6., ...,  2.,  0.,  6.],
       [ 1.,  4., 15., ...,  5.,  3.,  0.]])

## Sử dụng thuật Spectral Clustering

Xây dựng ma trận Similarity

In [4]:
similarity = (emails + emails.T)
similarity

array([[ 0., 31.,  8., ...,  8., 31.,  3.],
       [31.,  0., 11., ...,  3., 35., 12.],
       [ 8., 11.,  0., ...,  4.,  8., 31.],
       ...,
       [ 8.,  3.,  4., ...,  0.,  4., 10.],
       [31., 35.,  8., ...,  4.,  0.,  9.],
       [ 3., 12., 31., ..., 10.,  9.,  0.]])

Sử dụng Spectral Clustering với số cụm là 4 trên ma trận `Similarity`

In [5]:
spectral = SpectralClustering(n_clusters=4, affinity='precomputed')
spectral.fit(similarity)

SpectralClustering(affinity='precomputed', n_clusters=4)

In [6]:
labels = spectral.labels_

In [7]:
df = pd.DataFrame(labels, columns=['Cluster'])

In [8]:
df

,Cluster
0,3
1,3
2,0
3,0
4,2
...,...
95,1
96,2
97,1
98,3


In [9]:
df.value_counts()

Cluster
0          30
1          30
2          20
3          20
dtype: int64

Thuật Spectral Clustering phân 100 người vào 4 cụm với số lượng như trên. Tuy thuật toán này rất lợi hại nhưng vẫn cần có yếu tố con người để biết chính xác các cụm `0`, `1`, `2` và `3` là cụm của phòng ban nào :))